# Predict household income from satellite imagery data

First pass.

General ML pipeline steps:
1. Import data
2. Split data into test/train sets
3. Preprocess test/train sets separately
4. Generate features from data
5. For each regressor-hyperparameter combination:
    - Train regressor with given hyperparameters and training data and labels
    - Generate predicted labels for test data with trained regressor
    - Evaluate regressor-hyperparameter performance against actual test labels and get $R^2$
6. Explore best-performing models

In [1]:
import os
import math
import pickle
import numpy as np
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import LinearSVR
from sklearn.svm import LinearSVC

from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, RandomForestClassifier

from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt

# Import configuration file
import config as cf

# Display options 
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = -1

# Turn off big pink warnings
import warnings
warnings.filterwarnings('ignore')

# Data file path 
final_data_file_path = "/Users/robmarty/Dropbox/World Bank/IEs/Pakistan Poverty Estimation from Satellites/Data/FinalData"
#


In [2]:
# Test grid to make sure everything works - limited models and parameters
GRID_TEST_CLASS = {
    'regressors': ['LinearSVC','DecisionTreeClassifier', 'BaggingClassifier',
                    'GradientBoostingClassifier', 'RandomForestClassifier'],
    'LinearSVC': [
        {'penalty': penalty, 'C': C, 'loss': loss, 'max_iter': max_iter,
        'random_state': 0} \
        for penalty in ('l2', ) \
        for C in (1e-2, 1e2) \
        for loss in ('epsilon_insensitive', ) \
        for max_iter in (1e3, 1e5)
    ],
    'DecisionTreeClassifier': [
        {'criterion': criterion, 'splitter': splitter, 'max_depth': max_depth,
        'max_features': max_features, 'random_state': 0} \
        for criterion in ('gini', ) \
        for splitter in ('best', ) \
        for max_depth in (1, 5, 10, 20, 30) \
        for max_features in ('sqrt', ) \
    ],
    'BaggingClassifier': [
        {'n_estimators': n_estimators, 'max_features': max_features,
        'random_state': 0, 'n_jobs': -1} \
        for n_estimators in (100, 1000, 10000) \
        for max_features in (0.3, 0.5, 1.0)
    ],
    'GradientBoostingClassifier': [
        {'loss': loss, 'learning_rate': rate, 'n_estimators': n_estimators,
        'criterion': criterion, 'max_features': max_features,
        'random_state': 0} \
        for loss in ('deviance', ) \
        for rate in (1e-4, )
        for n_estimators in (100, ) \
        for criterion in ('friedman_mse', ) \
        for max_features in ('sqrt', ) \
    ],
    'RandomForestClassifier': [
        {'n_estimators': n_estimators, 'criterion': criterion,
        'max_depth': max_depth, 'max_features': max_features, 'n_jobs': -1,
        'random_state': 0} \
        for n_estimators in (10, 100, 1000) \
        for criterion in ('gini', ) \
        for max_depth in (1, ) \
        for max_features in ('sqrt', )
    ]
}

## 1. Import data and drop "future" rows

In [3]:
DATA_PATH = os.path.join(final_data_file_path, 'BISP','Merged Datasets', 'bisp_socioeconomic_satellite_firstdiff_r13.csv')
df = pd.read_csv(DATA_PATH)
df.shape

(3177, 478)

In [4]:
df.head()

,uid,survey_round,province,psu,locality,period,treatment,panel,present11,present13,present16,hh_size,income_last_month_N_NAs,income_last_month,income_last_year_N_NAs,income_last_year,pscores,b1_buff_0.1km_mean,b2_buff_0.1km_mean,b3_buff_0.1km_mean,b4_buff_0.1km_mean,b5_buff_0.1km_mean,b6_buff_0.1km_mean,b7_buff_0.1km_mean,b12_buff_0.1km_mean,b13_buff_0.1km_mean,b14_buff_0.1km_mean,b15_buff_0.1km_mean,b16_buff_0.1km_mean,b17_buff_0.1km_mean,b23_buff_0.1km_mean,b24_buff_0.1km_mean,b25_buff_0.1km_mean,b26_buff_0.1km_mean,b27_buff_0.1km_mean,b34_buff_0.1km_mean,b35_buff_0.1km_mean,b36_buff_0.1km_mean,b37_buff_0.1km_mean,b45_buff_0.1km_mean,b46_buff_0.1km_mean,b47_buff_0.1km_mean,b56_buff_0.1km_mean,b57_buff_0.1km_mean,b67_buff_0.1km_mean,b1_buff_0.5km_mean,b2_buff_0.5km_mean,b3_buff_0.5km_mean,b4_buff_0.5km_mean,b5_buff_0.5km_mean,b6_buff_0.5km_mean,b7_buff_0.5km_mean,b12_buff_0.5km_mean,b13_buff_0.5km_mean,b14_buff_0.5km_mean,b15_buff_0.5km_mean,b16_buff_0.5km_mean,b17_buff_0.5km_mean,b23_buff_0.5km_mean,b24_buff_0.5km_mean,b25_buff_0.5km_mean,b26_buff_0.5km_mean,b27_buff_0.5km_mean,b34_buff_0.5km_mean,b35_buff_0.5km_mean,b36_buff_0.5km_mean,b37_buff_0.5km_mean,b45_buff_0.5km_mean,b46_buff_0.5km_mean,b47_buff_0.5km_mean,b56_buff_0.5km_mean,b57_buff_0.5km_mean,b67_buff_0.5km_mean,b1_buff_1km_mean,b2_buff_1km_mean,b3_buff_1km_mean,b4_buff_1km_mean,b5_buff_1km_mean,b6_buff_1km_mean,b7_buff_1km_mean,b12_buff_1km_mean,b13_buff_1km_mean,b14_buff_1km_mean,b15_buff_1km_mean,b16_buff_1km_mean,b17_buff_1km_mean,b23_buff_1km_mean,b24_buff_1km_mean,b25_buff_1km_mean,b26_buff_1km_mean,b27_buff_1km_mean,b34_buff_1km_mean,b35_buff_1km_mean,b36_buff_1km_mean,b37_buff_1km_mean,b45_buff_1km_mean,b46_buff_1km_mean,b47_buff_1km_mean,b56_buff_1km_mean,b57_buff_1km_mean,b67_buff_1km_mean,b1_buff_1.5km_mean,b2_buff_1.5km_mean,b3_buff_1.5km_mean,b4_buff_1.5km_mean,b5_buff_1.5km_mean,b6_buff_1.5km_mean,b7_buff_1.5km_mean,b12_buff_1.5km_mean,b13_buff_1.5km_mean,b14_buff_1.5km_mean,b15_buff_1.5km_mean,b16_buff_1.5km_mean,b17_buff_1.5km_mean,b23_buff_1.5km_mean,b24_buff_1.5km_mean,b25_buff_1.5km_mean,b26_buff_1.5km_mean,b27_buff_1.5km_mean,b34_buff_1.5km_mean,b35_buff_1.5km_mean,b36_buff_1.5km_mean,b37_buff_1.5km_mean,b45_buff_1.5km_mean,b46_buff_1.5km_mean,b47_buff_1.5km_mean,b56_buff_1.5km_mean,b57_buff_1.5km_mean,b67_buff_1.5km_mean,b1_buff_2km_mean,b2_buff_2km_mean,b3_buff_2km_mean,b4_buff_2km_mean,b5_buff_2km_mean,b6_buff_2km_mean,b7_buff_2km_mean,b12_buff_2km_mean,b13_buff_2km_mean,b14_buff_2km_mean,b15_buff_2km_mean,b16_buff_2km_mean,b17_buff_2km_mean,b23_buff_2km_mean,b24_buff_2km_mean,b25_buff_2km_mean,b26_buff_2km_mean,b27_buff_2km_mean,b34_buff_2km_mean,b35_buff_2km_mean,b36_buff_2km_mean,b37_buff_2km_mean,b45_buff_2km_mean,b46_buff_2km_mean,b47_buff_2km_mean,b56_buff_2km_mean,b57_buff_2km_mean,b67_buff_2km_mean,b1_buff_0.1km_min,b2_buff_0.1km_min,b3_buff_0.1km_min,b4_buff_0.1km_min,b5_buff_0.1km_min,b6_buff_0.1km_min,b7_buff_0.1km_min,b12_buff_0.1km_min,b13_buff_0.1km_min,b14_buff_0.1km_min,b15_buff_0.1km_min,b16_buff_0.1km_min,b17_buff_0.1km_min,b23_buff_0.1km_min,b24_buff_0.1km_min,b25_buff_0.1km_min,b26_buff_0.1km_min,b27_buff_0.1km_min,b34_buff_0.1km_min,b35_buff_0.1km_min,b36_buff_0.1km_min,b37_buff_0.1km_min,b45_buff_0.1km_min,b46_buff_0.1km_min,b47_buff_0.1km_min,b56_buff_0.1km_min,b57_buff_0.1km_min,b67_buff_0.1km_min,b1_buff_0.5km_min,b2_buff_0.5km_min,b3_buff_0.5km_min,b4_buff_0.5km_min,b5_buff_0.5km_min,b6_buff_0.5km_min,b7_buff_0.5km_min,b12_buff_0.5km_min,b13_buff_0.5km_min,b14_buff_0.5km_min,b15_buff_0.5km_min,b16_buff_0.5km_min,b17_buff_0.5km_min,b23_buff_0.5km_min,b24_buff_0.5km_min,b25_buff_0.5km_min,b26_buff_0.5km_min,b27_buff_0.5km_min,b34_buff_0.5km_min,b35_buff_0.5km_min,b36_buff_0.5km_min,b37_buff_0.5km_min,b45_buff_0.5km_min,b46_buff_0.5km_min,b47_buff_0.5km_min,b56_buff_0.5km_min,b57_buff_0.5km_min,b67_buff_0.5km_min,b1_buff_1km_min,b2_buff_1km_min,b3_buff_1km_min,b4_buff_1km_min,b5_buff_1km_min,b6_buff_1km_min,b7_buff_1k

In [17]:
# DV as Quantiles
#df['pscores_2011'] = pd.qcut(df['pscores_2011'], 3, labels=False)
#df['pscores_2011'].value_counts()
df['pscores_bin'] = df['pscores'] < 0

In [18]:
#df = df.drop(['pscores_2011'], axis=1)

In [19]:
# Keep only 2011 columns, but include viirs_2012
## VIIRS Columns
df_viirs = df.filter(regex='viirs').filter(regex='_2km')
df_landsat = df.filter(regex='^b').filter(regex='_1km')
df_y = df.filter(regex='^pscores_bin$')

df_all = df_y.join(df_viirs).join(df_landsat)
df_all.head()

,pscores_bin,viirs_spatialmean_monthlymean_buff_2km,viirs_spatialmean_monthlysd_buff_2km,viirs_spatialmax_monthlymean_buff_2km,viirs_spatialmax_monthlysd_buff_2km,viirs_spatialmin_monthlymean_buff_2km,viirs_spatialmin_monthlysd_buff_2km,viirs_spatialsd_monthlymean_buff_2km,viirs_spatialsd_monthlysd_buff_2km,b1_buff_1km_mean,b2_buff_1km_mean,b3_buff_1km_mean,b4_buff_1km_mean,b5_buff_1km_mean,b6_buff_1km_mean,b7_buff_1km_mean,b12_buff_1km_mean,b13_buff_1km_mean,b14_buff_1km_mean,b15_buff_1km_mean,b16_buff_1km_mean,b17_buff_1km_mean,b23_buff_1km_mean,b24_buff_1km_mean,b25_buff_1km_mean,b26_buff_1km_mean,b27_buff_1km_mean,b34_buff_1km_mean,b35_buff_1km_mean,b36_buff_1km_mean,b37_buff_1km_mean,b45_buff_1km_mean,b46_buff_1km_mean,b47_buff_1km_mean,b56_buff_1km_mean,b57_buff_1km_mean,b67_buff_1km_mean,b1_buff_1km_min,b2_buff_1km_min,b3_buff_1km_min,b4_buff_1km_min,b5_buff_1km_min,b6_buff_1km_min,b7_buff_1km_min,b12_buff_1km_min,b13_buff_1km_min,b14_buff_1km_min,b15_buff_1km_min,b16_buff_1km_min,b17_buff_1km_min,b23_buff_1km_min,b24_buff_1km_min,b25_buff_1km_min,b26_buff_1km_min,b27_buff_1km_min,b34_buff_1km_min,b35_buff_1km_min,b36_buff_1km_min,b37_buff_1km_min,b45_buff_1km_min,b46_buff_1km_min,b47_buff_1km_min,b56_buff_1km_min,b57_buff_1km_min,b67_buff_1km_min,b1_buff_1km_max,b2_buff_1km_max,b3_buff_1km_max,b4_buff_1km_max,b5_buff_1km_max,b6_buff_1km_max,b7_buff_1km_max,b12_buff_1km_max,b13_buff_1km_max,b14_buff_1km_max,b15_buff_1km_max,b16_buff_1km_max,b17_buff_1km_max,b23_buff_1km_max,b24_buff_1km_max,b25_buff_1km_max,b26_buff_1km_max,b27_buff_1km_max,b34_buff_1km_max,b35_buff_1km_max,b36_buff_1km_max,b37_buff_1km_max,b45_buff_1km_max,b46_buff_1km_max,b47_buff_1km_max,b56_buff_1km_max,b57_buff_1km_max,b67_buff_1km_max
0,False,0.244809,-0.254202,1.153745,-0.229618,-0.016654,-0.017958,0.279187,-0.107712,-38.134660,-54.520337,-124.118536,126.720947,-98.793725,-30.003341,-184.361418,-0.001151,-0.024045,0.034539,0.000952,0.011668,-0.025312,-0.023822,0.040444,0.002119,0.014496,-0.026433,0.063285,0.024047,0.035483,-0.003654,-0.044497,-0.029081,-0.072213,0.015190,-0.029438,-0.042740,-3.0,59.0,23.0,189.0,77.5,-10.5,47.0,0.042585,0.019687,0.062731,0.035053,0.000501,0.037823,-0.023494,0.029189,-0.004201,-0.026960,-0.005241,0.049611,0.018174,-0.011457,0.018279,-0.035769,-0.063184,-0.033099,-0.029780,-0.000843,0.022360,19.5,-34.0,-179.5,102.0,-183.0,42.0,500.5,-0.014714,-0.042357,0.007604,-0.025551,0.000469,0.050267,-0.029070,0.022054,-0.014320,0.014673,0.068150,0.051360,0.012790,0.043953,0.099388,-0.039470,-0.008266,0.051623,0.031011,0.091273,0.059490
1,False,0.215588,-0.289247,-0.315197,-1.852972,0.049441,0.000172,0.140318,-0.289600,-33.108280,-43.913289,-111.013752,132.278083,-84.460046,-30.926896,-165.912710,0.000404,-0.022373,0.033712,0.001183,0.009684,-0.023583,-0.023674,0.038133,0.000974,0.010918,-0.026086,0.060768,0.022672,0.031519,-0.003462,-0.043124,-0.030655,-0.069330,0.012211,-0.027814,-0.038265,-3.0,49.0,23.0,189.0,28.0,-24.0,-2.0,0.036080,0.019687,0.062731,0.014610,-0.000724,0.001158,-0.016842,0.035245,-0.019898,-0.024125,-0.035452,0.049611,-0.003547,-0.012869,-0.018707,-0.058282,-0.065198,-0.065348,-0.013559,-0.014132,0.001484,106.5,46.5,213.5,287.0,367.0,-23.0,230.5,-0.023200,0.004681,0.006054,0.003657,-0.034493,-0.004590,0.028221,0.028802,0.024290,-0.014630,0.016162,0.002006,-0.000151,-0.043787,-0.010367,-0.002151,-0.045928,-0.012734,-0.042127,-0.010533,0.032766
2,False,0.183524,-0.291188,-0.315197,-1.852972,-0.016654,-0.017958,0.142993,-0.296380,-32.582630,-40.047134,-97.124964,124.080012,-75.249782,-30.361362,-140.287169,0.001815,-0.017179,0.031837,0.002501,0.009534,-0.017125,-0.019748,0.034897,0.001200,0.009598,-0.020488,0.053939,0.019339,0.026885,-0.001613,-0.039202,-0.028689,-0.060137,0.010325,-0.022375,-0.031328,-3.0,59.0,23.0,189.0,77.5,-11.0,47.0,0.042585,0.019687,0.062731,0.035053,0.000456,0.037823,-0.023494,0.029189,-0.004201,-0.027012,-0.005241,0.049611,0.018174,-0.011508,0.018279,-0.035769,-0.063254,-0.033099,-0.02

In [16]:
# Drop columns where the label is missing
#df = df.loc[~pd.isnull(df['hhinc_2011'])]

#df.shape

## 2. Split data into test/train

In [20]:
LABEL = 'pscores_bin'
TEST_SIZE = 0.3

# Separate feature sets from label sets
x_df = df_all.drop(labels=[LABEL], axis=1)
y_df = df_all[LABEL]

# Split into test and train sets for features and labels
x_train, x_test, y_train, y_test =  train_test_split(x_df, y_df, test_size=TEST_SIZE)

## 3. Preprocess data

All vars are numeric - impute missing data with mean

In [32]:
x_train.head()

,dmspols_2011,l7_2011_1,l7_2011_2,l7_2011_3,l7_2011_4,l7_2011_5,l7_2011_6,l7_2011_7,viirs_2012,uid,dmspols_2011_imputed,viirs_2012_imputed,ratio_1_2,ratio_1_3,ratio_1_4,ratio_1_5,ratio_1_6,ratio_1_7,ratio_2_3,ratio_2_4,ratio_2_5,ratio_2_6,ratio_2_7,ratio_3_4,ratio_3_5,ratio_3_6,ratio_3_7,ratio_4_5,ratio_4_6,ratio_4_7,ratio_5_6,ratio_5_7,ratio_6_7
374,24.0,901.910885,1276.550363,1411.094775,2996.116401,2546.972714,3031.464877,1907.712192,1.229348,4900320,0,0,0.171974,0.220139,0.537248,0.476984,0.541406,0.357984,0.050060,0.402457,0.332265,0.407360,0.198213,0.359643,0.286978,0.364738,0.149637,0.081028,0.005864,0.221950,0.086851,0.143503,0.227518
2388,31.0,1010.649942,1344.685760,1589.681990,2445.176189,2589.602233,3018.334252,2046.523202,4.102749,25703397,0,0,0.141821,0.222676,0.415104,0.438567,0.498310,0.338834,0.083492,0.290377,0.316427,0.383599,0.206958,0.212026,0.239256,0.310036,0.125637,0.028686,0.104907,0.088753,0.076451,0.117141,0.191873
3461,6.5,1016.671152,1320.956019,1479.582899,2359.837384,2234.348958,3026.724826,1685.984520,0.301120,37700984,0,0,0.130168,0.185443,0.397797,0.374553,0.497120,0.247650,0.056642,0.282244,0.256910,0.392340,0.121395,0.229268,0.203226,0.343328,0.065202,0.027315,0.123806,0.166555,0.150611,0.139877,0.284495
3662,0.0,617.041618,787.066415,809.077871,2105.904002,1763.348318,2938.720708,1204.052204,0.109788,39800512,0,0,0.121091,0.134656,0.546784,0.481563,0.652934,0.322340,0.013790,0.455877,0.382793,0.577503,0.209423,0.444883,0.370961,0.568238,0.196199,0.088533,0.165090,0.272466,0.249969,0.188480,0.418722
83,18.0,880.414847,1152.104803,1318.878894,2572.979330,2284.983406,3030.307132,1783.694032,1.105853,1702700,0,0,0.133672,0.199366,0.490116,0.443726,0.549743,0.339055,0.067493,0.381434,0.329604,0.449072,0.215134,0.322237,0.268075,0.393505,0.149816,0.059283,0.081618,0.181167,0.140223,0.123207,0.258956


In [33]:
x_test.head()

,dmspols_2011,l7_2011_1,l7_2011_2,l7_2011_3,l7_2011_4,l7_2011_5,l7_2011_6,l7_2011_7,viirs_2012,uid,dmspols_2011_imputed,viirs_2012_imputed,ratio_1_2,ratio_1_3,ratio_1_4,ratio_1_5,ratio_1_6,ratio_1_7,ratio_2_3,ratio_2_4,ratio_2_5,ratio_2_6,ratio_2_7,ratio_3_4,ratio_3_5,ratio_3_6,ratio_3_7,ratio_4_5,ratio_4_6,ratio_4_7,ratio_5_6,ratio_5_7,ratio_6_7
3145,6.000000,1067.911645,1347.568656,1496.182648,2448.161935,2328.447567,3004.758980,1737.813586,0.337368,34902063,0,0,0.115777,0.167026,0.392554,0.371143,0.475572,0.238762,0.052260,0.289956,0.266832,0.380760,0.126482,0.241353,0.217607,0.335169,0.074716,0.025063,0.102073,0.169697,0.126811,0.145252,0.267143
3888,0.000000,909.066599,1054.835316,1108.651480,1493.869704,1341.081399,2903.133198,1080.304266,0.173900,43301419,0,0,0.074224,0.098916,0.243370,0.191994,0.523075,0.086076,0.024875,0.172258,0.119472,0.466981,0.011928,0.148017,0.094880,0.447303,0.012950,0.053895,0.320505,0.160659,0.368043,0.107697,0.457602
1117,7.666667,843.911082,1138.947636,1333.943429,2591.361184,2285.129243,3031.391065,1782.293734,0.276674,14502621,0,0,0.148794,0.225007,0.508679,0.460594,0.564467,0.357315,0.078853,0.389355,0.334742,0.453787,0.220230,0.320336,0.262826,0.388847,0.143876,0.062798,0.078259,0.184987,0.140367,0.123625,0.259489
3348,10.000000,1188.818870,1759.164822,2251.561736,3211.930984,3842.226267,3058.202097,3118.564065,0.925885,36901045,0,0,0.193470,0.308903,0.459720,0.527407,0.440163,0.448009,0.122770,0.292243,0.371883,0.269657,0.278695,0.175779,0.261031,0.151916,0.161449,0.089351,0.024518,0.014749,0.113620,0.103963,0.009772
5245,2.000000,1543.693146,2214.106303,2833.584665,3405.777374,4622.612692,3047.386146,3561.186030,0.158197,61200110,0,0,0.178406,0.294679,0.376219,0.499313,0.327525,0.395209,0.122725,0.212046,0.352290,0.158373,0.233249,0.091707,0.239938,0.036355,0.113781,0.151567,0.055537,0.022307,0.205375,0.129699,0.077747


In [34]:
# check that lengths match
print(len(x_train) == len(y_train))
print(len(x_test) == len(y_test))

True
True


In [34]:
DAY_FEATURES = df_all.filter(regex='^b', axis=1).columns.tolist()
NIGHT_FEATURES = df_all.filter(regex='viirs', axis=1).columns.tolist()
ALL_FEATURES = DAY_FEATURES + NIGHT_FEATURES

print("Day-only:", DAY_FEATURES)
print("-----")
print("Night-only:", NIGHT_FEATURES)

Day-only: ['b1_buff_1km_mean', 'b2_buff_1km_mean', 'b3_buff_1km_mean', 'b4_buff_1km_mean', 'b5_buff_1km_mean', 'b6_buff_1km_mean', 'b7_buff_1km_mean', 'b12_buff_1km_mean', 'b13_buff_1km_mean', 'b14_buff_1km_mean', 'b15_buff_1km_mean', 'b16_buff_1km_mean', 'b17_buff_1km_mean', 'b23_buff_1km_mean', 'b24_buff_1km_mean', 'b25_buff_1km_mean', 'b26_buff_1km_mean', 'b27_buff_1km_mean', 'b34_buff_1km_mean', 'b35_buff_1km_mean', 'b36_buff_1km_mean', 'b37_buff_1km_mean', 'b45_buff_1km_mean', 'b46_buff_1km_mean', 'b47_buff_1km_mean', 'b56_buff_1km_mean', 'b57_buff_1km_mean', 'b67_buff_1km_mean', 'b1_buff_1km_min', 'b2_buff_1km_min', 'b3_buff_1km_min', 'b4_buff_1km_min', 'b5_buff_1km_min', 'b6_buff_1km_min', 'b7_buff_1km_min', 'b12_buff_1km_min', 'b13_buff_1km_min', 'b14_buff_1km_min', 'b15_buff_1km_min', 'b16_buff_1km_min', 'b17_buff_1km_min', 'b23_buff_1km_min', 'b24_buff_1km_min', 'b25_buff_1km_min', 'b26_buff_1km_min', 'b27_buff_1km_min', 'b34_buff_1km_min', 'b35_buff_1km_min', 'b36_buff_1km_m

## 5. Train and Evaluate Regressors

### 5.1 Training

In [35]:
x_all = x_test.append(x_train)
y_all = y_test.append(y_train)

In [36]:
# Define a TrainedRegressor object to hold key results information
class TrainedRegressor:
    
    def __init__(self, method, params, features, regressor):
        self.method = method
        self.params = params
        self.regressor = regressor
        self.features = features
    
    def __repr__(self):
        return f'Trained {self.method} on feature set {self.features} with params {self.params}'

In [37]:
# Use GRID_MAIN for full grid search
# parameters = cf.GRID_TEST_CLASS
parameters = GRID_TEST_CLASS

results_df = pd.DataFrame()
results_df_all = pd.DataFrame()
results_df_trainedonly_all = pd.DataFrame()

x_trainedonly_all = x_all.copy()

trained_list = []
trained_list_all = []
count = 0
# print('Training model ', end='')
for i in parameters['regressors']:
    for j in parameters[i]:
        for k in ('DAY_FEATURES', 'NIGHT_FEATURES', 'ALL_FEATURES'):
        
            print(f'Model {count}: Training {i} on {k} with params {str(j)}')

            # A. Train ------------------------------------
            # Initialize regressor, fit data, then append TrainedRegressor object to list
            # 1. Train Data
            regressor = eval(i)(**j)
            trained = regressor.fit(x_train[eval(k)], y_train)
            trained_list.append(TrainedRegressor(i, str(j), k, trained))

            # 2. All Data
            trained_all = regressor.fit(x_all[eval(k)], y_all)
            trained_list_all.append(TrainedRegressor(i, str(j), k, trained_all))
            
            
            
            
            
            
            
            # B. Results -------------------------------------
            # 1. Trained Model on Test Data - - - - - - - - - -
            pred_labels = trained_list[count].regressor.predict(x_test[eval(k)])

            pred_dict = {
                'regressor': trained_list[count].method,
                'features': trained_list[count].features,
                'params': trained_list[count].params,
                'accuracy_score': accuracy_score(y_true=y_test, y_pred=pred_labels)        
            }
    
            results_df = results_df.append(pred_dict, ignore_index=True) \
                .sort_values(by='accuracy_score', ascending=False, axis=0) \
                [['regressor', 'params', 'features', 'accuracy_score']]
        
            results_df.to_csv("/Users/robmarty/Desktop/pov_results_r13.csv")
            
            x_test['y_true'] = y_test
            x_test['y_predict_' + str(count)] = pred_labels
            x_test.to_csv(os.path.join(final_data_file_path, 'Data with Predicted Income', 'pov_opm_data_with_predictions_traineddatamodel_testdatapredict_r13.csv'))
            
            
            
            
            
            
            
            # 2. Trained All Model on All Data - - - - - - - - - -
            pred_labels_all = trained_list_all[count].regressor.predict(x_all[eval(k)])

            # Append results to dataframe and sort by R^2
            pred_dict = {
                'regressor': trained_list_all[count].method,
                'features': trained_list_all[count].features,
                'params': trained_list_all[count].params,
                'accuracy_score': accuracy_score(y_true=y_all, y_pred=pred_labels_all)        
            }
    
            results_df_all = results_df_all.append(pred_dict, ignore_index=True) \
                .sort_values(by='accuracy_score', ascending=False, axis=0) \
                [['regressor', 'params', 'features', 'accuracy_score']]
        
            results_df_all.to_csv("/Users/robmarty/Desktop/pov_results_all_r13.csv")

            # ALL
            x_trainedonly_all['y_true'] = y_all
            x_trainedonly_all['y_predict_' + str(count)] = trained_list_all[count].regressor.predict(x_all[eval(k)])
            x_trainedonly_all.to_csv(os.path.join(final_data_file_path, 'Data with Predicted Income', 'pov_opm_data_with_predictions_alldatamodel_alldatapredict_r13.csv'))
            
            
            
            
            
            
            # 3. Trained Model on All Data - - - - - - - - - -
            pred_labels_trainedonly_all = trained_list[count].regressor.predict(x_all[eval(k)])

            # Append results to dataframe and sort by R^2
            pred_dict = {
                'regressor': trained_list[count].method,
                'features': trained_list[count].features,
                'params': trained_list[count].params,
                'accuracy_score': accuracy_score(y_true=y_all, y_pred=pred_labels_trainedonly_all)        
            }
    
            results_df_trainedonly_all = results_df_trainedonly_all.append(pred_dict, ignore_index=True) \
                .sort_values(by='accuracy_score', ascending=False, axis=0) \
                [['regressor', 'params', 'features', 'accuracy_score']]
        
            results_df_trainedonly_all.to_csv("/Users/robmarty/Desktop/pov_results_trainedonly_all_r13.csv")

            # ALL
            x_all['y_true'] = y_all
            x_all['y_predict_' + str(count)] = trained_list[count].regressor.predict(x_all[eval(k)])
            x_all.to_csv(os.path.join(final_data_file_path, 'Data with Predicted Income', 'pov_opm_data_with_predictions_testdatamodel_alldatapredict_r13.csv'))

            ####
            count += 1


Model 0: Training LinearSVC on DAY_FEATURES with params {'penalty': 'l2', 'C': 0.01, 'loss': 'epsilon_insensitive', 'max_iter': 1000.0, 'random_state': 0}
Model 1: Training LinearSVC on NIGHT_FEATURES with params {'penalty': 'l2', 'C': 0.01, 'loss': 'epsilon_insensitive', 'max_iter': 1000.0, 'random_state': 0}
Model 2: Training LinearSVC on ALL_FEATURES with params {'penalty': 'l2', 'C': 0.01, 'loss': 'epsilon_insensitive', 'max_iter': 1000.0, 'random_state': 0}
Model 3: Training LinearSVC on DAY_FEATURES with params {'penalty': 'l2', 'C': 0.01, 'loss': 'epsilon_insensitive', 'max_iter': 100000.0, 'random_state': 0}
Model 4: Training LinearSVC on NIGHT_FEATURES with params {'penalty': 'l2', 'C': 0.01, 'loss': 'epsilon_insensitive', 'max_iter': 100000.0, 'random_state': 0}
Model 5: Training LinearSVC on ALL_FEATURES with params {'penalty': 'l2', 'C': 0.01, 'loss': 'epsilon_insensitive', 'max_iter': 100000.0, 'random_state': 0}
Model 6: Training LinearSVC on DAY_FEATURES with params {'pe

KeyboardInterrupt: 

In [ ]:
#y_test
#pred_labels
#parameters

### 5.2 Prediction and Evaluation

In [ ]:
results_df = pd.DataFrame()
for i in trained_list:
    
    # Get predicted results from test data
    features = eval(i.features)
    pred_labels = i.regressor.predict(x_test[features])
    
    # Append results to dataframe and sort by R^2
    pred_dict = {
        'regressor': i.method,
        'features': i.features,
        'params': i.params,
        'r2': r2_score(y_true=y_test, y_pred=pred_labels)        
    }
    
    results_df = results_df.append(pred_dict, ignore_index=True) \
        .sort_values(by='r2', ascending=False, axis=0) \
        [['regressor', 'params', 'features', 'r2']]

results_df.shape

In [ ]:
results_df